<a href="https://colab.research.google.com/github/hdtruong802/Pytorch-Assignments/blob/main/Pytorch_Image_Classification_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bài tập lập trình: Thực hành lập trình phân loại hình ảnh
```
ProtonX - Pytorch Class
```

### Hướng dẫn làm bài
- Trong bài tập này bạn sẽ sử dụng Python 3.
- Sau khi bạn viết Code của mình xong, hãy chạy dòng Code đó để xem kết quả bên dưới.

### [Quan trọng] Chú ý
- **Không sử dụng hàm `input()` tại bất kỳ dòng lệnh nào**
- **Không thay đổi dòng code return của hàm**

Các bạn sẽ thực hiện `code` trong các phần hiển thị `#TODO: Lập trình tại đây` và thay thế các vị trí `None`. Có những câu hỏi chỉ cần trả về đáp án.

Sau khi viết xong Code của bạn, bạn hãy ấn "SHIFT"+"ENTER" để thực hiện chạy lệnh của Cell đó.

---
Điểm số:
* 10 điểm / Câu

Tiêu chí chấm điểm:
* Các bài tập sẽ được chấm dựa trên các Test-case.
* Các bạn không khởi tạo lại giá trị đầu vào bên trong hàm. Có thể khởi tạo các giá trị này ngoài hàm nhằm mục đích kiểm thử.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 1. Mô hình CNN đơn giản
Kiến trúc mô hình CNN với
- Phép tích chập với 64 bộ lọc cỡ (3, 3)
- Phép maxpool với cỡ bộ lọc (2, 2) bước nhảy 2
- Mô hình phân loại đầu ra là 10

In [ ]:
# Thiết kế kiến trúc mạng CNN đơn giản
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # TODO 1: Định nghĩa phép tích chập
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        # TODO 2: Định nghĩa phép RELU
        self.relu = nn.ReLU()
        # TODO 3: Định nghĩa phép Maxpool
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # TODO 4: Định nghĩa lớp nhân tuyến tính
        self.fc1 = nn.Linear(64 * 16 * 16,10)

    def forward(self, x):
        # TODO 5: Tính toán đầu ra của lớp tích chập
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        # TODO 6: Tiến hành duỗi feature map
        x = x.view(-1, 64 * 16 * 16)
        # TODO 7: Đưa qua lớp nhân tuyến tính
        x = self.fc1(x)
        return x

**Code đánh giá lập trình mô hình**

In [ ]:
import torch
import unittest

class TestSimpleCNN(unittest.TestCase):
    def setUp(self):
        self.model = SimpleCNN()

    def test_forward_pass(self):
        # Kiểm thử forward pass với input ngẫu nhiên
        random_input = torch.randn(1, 3, 32, 32)  # Giả sử kích thước input là (batch_size, channels, height, width)
        output = self.model(random_input)
        self.assertEqual(output.shape, (1, 10))  # Điều chỉnh kích thước output dựa trên kiến trúc của mô hình

    def test_conv_layer(self):
        # Kiểm thử sự hiện diện của layer convolutional trong mô hình
        conv_layers = [module for module in self.model.modules() if isinstance(module, torch.nn.Conv2d)]
        self.assertTrue(len(conv_layers) > 0)

    def test_relu_layer(self):
        # Kiểm thử sự hiện diện của layer kích hoạt ReLU trong mô hình
        relu_layers = [module for module in self.model.modules() if isinstance(module, torch.nn.ReLU)]
        self.assertTrue(len(relu_layers) > 0)

    def test_pooling_layer(self):
        # Kiểm thử sự hiện diện của layer pooling trong mô hình
        pool_layers = [module for module in self.model.modules() if isinstance(module, torch.nn.MaxPool2d)]
        self.assertTrue(len(pool_layers) > 0)

    def test_fc_layer(self):
        # Kiểm thử sự hiện diện của layer fully connected trong mô hình
        fc_layers = [module for module in self.model.modules() if isinstance(module, torch.nn.Linear)]
        self.assertTrue(len(fc_layers) > 0)

    def test_output_size(self):
        # Kiểm thử kích thước output của layer fully connected
        dummy_input = torch.randn(1, 3, 32, 32)  # Giả sử kích thước input là (batch_size, channels, height, width)
        output = self.model(dummy_input)
        expected_output_size = (1, 10)  # Điều chỉnh kích thước output dựa trên kiến trúc của mô hình
        self.assertEqual(output.shape, expected_output_size)

    def test_conv1_weights_shape(self):
        # Kiểm thử hình dạng của trọng số layer conv1
        expected_shape = (64, 3, 3, 3)  # (out_channels, in_channels, kernel_size[0], kernel_size[1])
        actual_shape = tuple(self.model.conv1.weight.shape)
        self.assertEqual(expected_shape, actual_shape)

    def test_conv1_forward_pass(self):
        # Kiểm thử forward pass qua layer conv1
        input_tensor = torch.randn(1, 3, 32, 32)  # Batch size là 1, 3 channels, kích thước hình ảnh 32x32
        output_tensor = self.model.conv1(input_tensor)
        self.assertEqual(output_tensor.shape[1], 64)  # Kiểm tra số lượng channels đầu ra
        self.assertEqual(output_tensor.shape[2], 32)  # Kiểm tra chiều cao của tensor đầu ra
        self.assertEqual(output_tensor.shape[3], 32)  # Kiểm tra chiều rộng của tensor đầu ra

    def test_train_on_batch(self):
        # Kiểm thử việc huấn luyện trên một batch với input và nhãn ngẫu nhiên
        random_input = torch.randn(32, 3, 32, 32)  # Giả sử batch size là 32
        random_labels = torch.randint(0, 10, (32,))  # Giả sử có 10 classes
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)

        # Bước huấn luyện
        optimizer.zero_grad()
        outputs = self.model(random_input)
        loss = criterion(outputs, random_labels)
        loss.backward()
        optimizer.step()

        # Kiểm tra rằng loss là một tensor hợp lệ
        self.assertIsInstance(loss, torch.Tensor)

        # Kiểm tra rằng giá trị loss nằm trong khoảng chấp nhận được (bạn có thể điều chỉnh ngưỡng)
        self.assertTrue(0 <= loss.item() < 10)



suite = unittest.TestLoader().loadTestsFromTestCase(TestSimpleCNN)
unittest.TextTestRunner(verbosity=2).run(suite)

test_conv1_forward_pass (__main__.TestSimpleCNN) ... ok
test_conv1_weights_shape (__main__.TestSimpleCNN) ... ok
test_conv_layer (__main__.TestSimpleCNN) ... ok
test_fc_layer (__main__.TestSimpleCNN) ... ok
test_forward_pass (__main__.TestSimpleCNN) ... ok
test_output_size (__main__.TestSimpleCNN) ... ok
test_pooling_layer (__main__.TestSimpleCNN) ... ok
test_relu_layer (__main__.TestSimpleCNN) ... ok
test_train_on_batch (__main__.TestSimpleCNN) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.236s

OK


<unittest.runner.TextTestResult run=9 errors=0 failures=0>

Kết quả mong đợi

```python
test_conv1_forward_pass (__main__.TestSimpleCNN) ... ok
test_conv1_weights_shape (__main__.TestSimpleCNN) ... ok
test_conv_layer (__main__.TestSimpleCNN) ... ok
test_fc_layer (__main__.TestSimpleCNN) ... ok
test_forward_pass (__main__.TestSimpleCNN) ... ok
test_output_size (__main__.TestSimpleCNN) ... ok
test_pooling_layer (__main__.TestSimpleCNN) ... ok
test_relu_layer (__main__.TestSimpleCNN) ... ok
test_train_on_batch (__main__.TestSimpleCNN) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.058s

OK
<unittest.runner.TextTestResult run=9 errors=0 failures=0>
```

In [ ]:
# TODO 8: Lập trình hàm transform
# Thêm một số chuẩn hóa bất kỳ
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Tải về và tiền xử lý bộ dữ liệu CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Khởi tạo Dataloader với tập train
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Khởi tạo Dataloader với tập train
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Khởi tạo mô hình
model = SimpleCNN()

100%|██████████| 170498071/170498071 [00:01<00:00, 97617170.13it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [ ]:
model.to(device)

SimpleCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=10, bias=True)
)

Kiến trúc của mô hình

In [ ]:
from torchsummary import summary
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
         MaxPool2d-3           [-1, 64, 16, 16]               0
            Linear-4                   [-1, 10]         163,850
Total params: 165,642
Trainable params: 165,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.13
Params size (MB): 0.63
Estimated Total Size (MB): 1.77
----------------------------------------------------------------


In [ ]:
# TODO 9: Khởi tạo hàm mất mát
criterion = nn.CrossEntropyLoss()
# TODO 10: Cài đặt thuật toán Adam
optimizer = optim.AdamW(
    params=model.parameters(),
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-8,
    weight_decay=0.01
)

num_epochs = 5

for epoch in range(num_epochs):
    # Training loop
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        total_val_samples = 0
        for val_images, val_labels in test_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)
            total_val_loss += val_loss.item() * val_labels.size(0)
            total_val_samples += val_labels.size(0)

        average_val_loss = total_val_loss / total_val_samples
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {average_val_loss:.4f}')

Epoch [1/5], Training Loss: 0.4849, Validation Loss: 1.0777
Epoch [2/5], Training Loss: 0.4338, Validation Loss: 1.0919
Epoch [3/5], Training Loss: 0.6741, Validation Loss: 1.0946
Epoch [4/5], Training Loss: 0.6556, Validation Loss: 1.1427
Epoch [5/5], Training Loss: 0.1925, Validation Loss: 1.1303


Kết quả mong đợi (Tương đối):
```python
Epoch [1/5], Training Loss: 0.6830, Validation Loss: 1.0610
Epoch [2/5], Training Loss: 1.3805, Validation Loss: 1.0449
Epoch [3/5], Training Loss: 0.5339, Validation Loss: 1.0223
Epoch [4/5], Training Loss: 0.2034, Validation Loss: 1.0707
Epoch [5/5], Training Loss: 0.3317, Validation Loss: 1.0935
```

In [ ]:

model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {100 * accuracy:.2f}%')


Test Accuracy: 66.51%


Kết quả mong đợi:

`Test Accuracy lớn hơn 66%`

### 2. Lập trình mô hình VGG
Để cải thiện chất lượng trên tập dữ liệu này, chúng ta sẽ sửa dụng Pretrained với Pytorch. Mô hình sử dụng sẽ là VGG 16, dùng toàn lại bộ tham số chỉ thay duy nhất lớp phân loại cuối cùng theo số lượng nhãn của bài toán.

Lập trình lớp VGG16Trainer với các hàm sau
- Hàm khởi tạo:
    - Tham số:
        - trainloader: Data train
        - testloader: Data test
        - num_classes=10
        - lr: Tốc độ học
        - momentum: Quán trính cho thuật toán SGD với Momentum
        - num_epochs: Số lượng epoch
- Hàm train
    - Lặp qua các epoch và training mô hình
- Hàm evaluate
    - Đánh giá chất lượng hiện tại của mô hình

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models

class VGG16Trainer:
    def __init__(self, trainloader, testloader, num_classes=10, lr=0.001, momentum=0.9, num_epochs=20):
        self.trainloader = trainloader
        self.testloader = testloader
        self.num_epochs = num_epochs
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # TODO 11: Load pre train VGG 16
        self.model = models.vgg16(pretrained=True)
        # TODO 12: Thay thế lớp phân loại cuối cùng của mô hình
        self.model.classifier[-1] = torch.nn.Linear(self.model.classifier[-1].in_features, num_classes)

        self.model.to(self.device)
        # self.model = None

        self.criterion = torch.nn.CrossEntropyLoss()
        # TODO 13: Cài đặt thuật toán SGD với momentum
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.0001, momentum=momentum, weight_decay=0.0005)

    def train(self):
        # Tiến hành training
        self.model.train()
        for epoch in range(self.num_epochs):
            running_loss = 0.0
            for i, data in enumerate(self.trainloader, 0):
                inputs, labels = data[0].to(self.device), data[1].to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()
            if epoch % 2 == 1:
                print(f"{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}")
                running_loss = 0.0

        print('Finished Training')
        # Lưu mô hình
        PATH = './cifar_vgg16.pth'
        torch.save(self.model.state_dict(), PATH)

    def evaluate(self):
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in self.testloader:
                images, labels = data[0].to(self.device), data[1].to(self.device)
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f'Độ chính xác mô hình: {accuracy}%')
        return accuracy

trainer = VGG16Trainer(train_loader, test_loader, num_classes=10, lr=0.001, momentum=0.9, num_epochs=20)

# TODO 14: Tiến hành đào tạo mô hình
trainer.train()

2, 782] loss: 0.279
4, 782] loss: 0.208
6, 782] loss: 0.173
8, 782] loss: 0.146
10, 782] loss: 0.126
12, 782] loss: 0.108
14, 782] loss: 0.093
16, 782] loss: 0.079
18, 782] loss: 0.066
20, 782] loss: 0.055
Finished Training


In [ ]:
# Đánh giá chất lượng mô hình
trainer.evaluate()

Độ chính xác mô hình: 85.4%


85.4

Kết quả mong đợi:

`Test Accuracy lớn hơn 85%`